In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

PATH = "data/"


# Normailze the data over x,y and z

In [2]:
def compute_x_y_z_mean_and_std(array):
    """Returns list of means and stds for each channel of the array."""
    means = []
    stds = []
    for i in range(3):
        channel = array[:,:,i]
        mean = np.mean(channel)
        std = np.std(channel)
        means.append(mean)
        stds.append(std)
    return means, stds


## Compute the dataset mean and std

In [3]:
general_mean = np.empty((1,3))
general_std = np.empty((1,3))

for folder in os.listdir(PATH):
    for file in os.listdir(PATH+folder):
        array = np.load(PATH+folder+"/"+file, allow_pickle=True)
        array = np.clip(array, -10, 10)
        means, stds = compute_x_y_z_mean_and_std(array)
        general_mean = np.append(general_mean, np.array([means]), axis=0)
        general_std = np.append(general_std, np.array([stds]), axis=0)
        # print(folder, file, means, stds)

dataset_mean = np.nanmean(general_mean, axis=0)
dataset_std = np.nanstd(general_std, axis=0)


In [4]:
general_std

array([[0.00000000e+000, 1.32038950e-311, 0.00000000e+000],
       [9.94667958e-002, 3.55457854e-001, 1.88731478e-001],
       [1.70998945e-001, 3.48339452e-001, 1.37660975e-001],
       [1.40657750e-001, 3.05853073e-001, 1.32741264e-001],
       [1.71437179e-001, 3.42695885e-001, 1.88092242e-001],
       [1.60883992e-001, 3.15836185e-001, 1.37447132e-001],
       [1.58759836e-001, 3.52427005e-001, 1.59663127e-001],
       [2.28214156e-001, 3.77174644e-001, 1.89980709e-001],
       [1.77607109e-001, 3.30737086e-001, 1.42986202e-001],
       [1.36107215e-001, 3.48227022e-001, 1.83390277e-001],
       [1.84334273e-001, 3.72180160e-001, 1.68591201e-001],
       [1.52823033e-001, 3.12916589e-001, 1.24395230e-001],
       [2.02880749e-001, 3.92977509e-001, 1.92105367e-001],
       [1.92944947e-001, 3.81183652e-001, 1.54981471e-001],
       [1.31778797e-001, 3.06254332e-001, 1.32971832e-001],
       [1.61158503e-001, 3.27881835e-001, 1.70968550e-001],
       [8.19819493e-002, 3.02941925e-001

# Neural Net

## Dataloader

In [5]:
for folder in os.listdir(PATH):
    print(folder)

come
lie down
sit
stay


In [6]:
# Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, data_folder, dataset_mean, dataset_std):
        self.dataset = []
        self.labels = []
        label_dict = {"come":0,"lie down":1,"sit":2,"stay":3}
        
        for folder in os.listdir(data_folder):
            for file in os.listdir(data_folder + folder):
                data = np.load(os.path.join(data_folder, folder, file))
                # normalize data
                data[:,:,0] = (data[:,:,0] - dataset_mean[0])/dataset_std[0]
                data[:,:,1] = (data[:,:,1] - dataset_mean[1])/dataset_std[1]
                data[:,:,2] = (data[:,:,2] - dataset_mean[2])/dataset_std[2]
                self.dataset.append(data)
                self.labels.append(label_dict[folder])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        data = self.dataset[index]
        data = np.transpose(data, (2, 0, 1))
        label = self.labels[index]

        return torch.tensor(data).float(), torch.tensor(label)


In [7]:
test = torch.zeros(16, 3, 18,5)
maxpool = nn.MaxPool2d(2)(test)
maxpool.shape

torch.Size([16, 3, 9, 2])

## Model

In [8]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=4):
        super().__init__()
    
        # input = 75x20x3
        self.conv1 = nn.Conv2d(n_input_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # input = 37x10x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # input = 18x5x64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # input = 9x2x128
        self.fc1 = nn.Linear(128*9*2, 512)
        self.fc2 = nn.Linear(512, n_output)

    
    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2) #maxpool of kernel size 2 to reduce the size of the images by a factor 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 128*9*2) #flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [26]:
# Set the necessary hyperparameters
batch_size = 4
# learning_rate = 0.001
num_epochs = 20
num_classes = 4

# Set the path to your data folder
data_folder = PATH  # Replace with the actual path to your data folder

# Create an instance of the dataset
dataset = CustomDataset(data_folder, dataset_mean, dataset_std)

# Creating data indices for training and validation splits:
validation_split = .2
shuffle_dataset = True
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


# Specify the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Create an instance of the CNN model and move it to the device
model = ConvNet().to(device)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.00001)

cuda


In [27]:
torch.cuda.get_device_properties(device)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4070 Ti', major=8, minor=9, total_memory=12281MB, multi_processor_count=60)

In [28]:
# Training loop
model.train()
total_step = len(train_loader)

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    num_correct = 0
    running_epoch_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred = outputs.argmax(dim = -1)

        num_correct += (y_pred == labels).sum().item()
        running_epoch_loss += loss.item()

    epoch_loss = running_epoch_loss / len(train_loader)
    epoch_acc = num_correct / len(train_loader.dataset)

    print(f'Training => Loss: {epoch_loss:.4f} | Train Accuracy: {epoch_acc:.4f}')

Epoch 1/20
Training => Loss: 1.2624 | Train Accuracy: 0.2966
Epoch 2/20
Training => Loss: 0.7586 | Train Accuracy: 0.6695
Epoch 3/20
Training => Loss: 0.5290 | Train Accuracy: 0.6949
Epoch 4/20
Training => Loss: 0.3645 | Train Accuracy: 0.7373
Epoch 5/20
Training => Loss: 0.3649 | Train Accuracy: 0.7119
Epoch 6/20
Training => Loss: 0.2280 | Train Accuracy: 0.7797
Epoch 7/20
Training => Loss: 0.2059 | Train Accuracy: 0.7627
Epoch 8/20
Training => Loss: 0.1656 | Train Accuracy: 0.7712
Epoch 9/20
Training => Loss: 0.1274 | Train Accuracy: 0.7881
Epoch 10/20
Training => Loss: 0.0923 | Train Accuracy: 0.7881
Epoch 11/20
Training => Loss: 0.0520 | Train Accuracy: 0.8051
Epoch 12/20
Training => Loss: 0.0430 | Train Accuracy: 0.8051
Epoch 13/20
Training => Loss: 0.0370 | Train Accuracy: 0.8051
Epoch 14/20
Training => Loss: 0.0345 | Train Accuracy: 0.8051
Epoch 15/20
Training => Loss: 0.0185 | Train Accuracy: 0.8051
Epoch 16/20
Training => Loss: 0.0157 | Train Accuracy: 0.8051
Epoch 17/20
Train

In [29]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs , labels in validation_loader:
        # Move data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted labels
        y_pred = outputs.argmax(dim = -1)

        # Update total and correct predictions
        total += labels.size(0)
        correct += (y_pred == labels).sum().item()

    # Print accuracy
    print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

# Save the trained model
torch.save(model.state_dict(), 'trained_model.pth')

Test Accuracy: 86.96%
